In [11]:
from aip import AipFace
import base64
import glob
APP_ID = '20201374'
API_KEY = 'XX'
SECRET_KEY = 'XX'
client = AipFace(APP_ID, API_KEY, SECRET_KEY)
for fname in glob.glob("C:/Users/Administrator/Desktop/face/pengcheng/*",recursive=True):
    with open(fname,"rb") as f:
        base64_data = base64.b64encode(f.read())
        image = str(base64_data, 'utf-8')
        imageType = "BASE64"
        groupId="Actor"
        #userId=str("胡歌").encode('utf-8')
        userId="pengcheng"
        options = {}
        options["user_info"] = "user's info"
        options["quality_control"] = "NORMAL"
        options["liveness_control"] = "HIGN"
        client.addUser(image, imageType, groupId, userId)

In [15]:
from aip import AipFace
import base64
import json
import glob
APP_ID = '20201374'
API_KEY = 'XX'
SECRET_KEY = 'XX'
client = AipFace(APP_ID, API_KEY, SECRET_KEY)
for fname in glob.glob("C:/Users/Administrator/Desktop/test/*",recursive=True):
    #print(fname)
    with open(fname,"rb") as f:
        base64_data = base64.b64encode(f.read())
        image = str(base64_data, 'utf-8')
        imageType = "BASE64"
        groupId="Actor"
        #userId="huge"
        #client.addUser(image, imageType, groupId, userId) 
        data=client.search(str(image), imageType, groupId)
        #print(data)
        if data["error_code"]==0:
            print("初步识别为 "+data["result"]["user_list"][0]["user_id"]," 准确度为 "+str(data["result"]["user_list"][0]["score"])+" "+fname)
        else:
            print("图片未识别",fname)

初步识别为 pengcheng  准确度为 94.462356567383 C:/Users/Administrator/Desktop/face/new\1.jpg
初步识别为 pengcheng  准确度为 94.462356567383 C:/Users/Administrator/Desktop/face/new\10 (10).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (103).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (104).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (105).jpg
初步识别为 zhangge  准确度为 90.448204040527 C:/Users/Administrator/Desktop/face/new\10 (107).jpg
初步识别为 pengcheng  准确度为 62.38249206543 C:/Users/Administrator/Desktop/face/new\10 (11).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (110).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (111).jpg
初步识别为 liuyaren  准确度为 26.887546539307 C:/Users/Administrator/Desktop/face/new\10 (112).jpg
初步识别为 pengcheng  准确度为 90.964920043945 C:/Users/Administrator/Desktop/face/new\10 (113).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (17).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (21).jpg
图片未识别 C:/Users/Administrator/Desktop/face/new\10 (26).jp

In [6]:
"""使用百度云接口和人脸库完成本地合影图片的多人脸识别"""
from aip import AipFace
from aip import AipFace
import base64
import dlib
import matplotlib.pyplot as plt
import numpy as np
import os, math,cv2
from skimage import io

"""
人脸特征点检测
"""
def rect_to_bb(rect): # 获得人脸矩形的坐标信息
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

"""
人脸对齐
"""
def face_alignment(faces):
    predictor = dlib.shape_predictor("H:/JupyterNotebook/shape_predictor_68_face_landmarks.dat") # 用来预测关键点
    faces_aligned = []
    for face in faces:
        rec = dlib.rectangle(0,0,face.shape[0],face.shape[1])
        shape = predictor(np.uint8(face),rec) # 注意输入的必须是uint8类型     
        order = [36,45,30,48,54] # left eye, right eye, nose, left mouth, right mouth  注意关键点的顺序，这个在网上可以找
        for j in order:
            x = shape.part(j).x
            y = shape.part(j).y
        eye_center =((shape.part(36).x + shape.part(45).x) * 1./2, # 计算两眼的中心坐标
                      (shape.part(36).y + shape.part(45).y) * 1./2)
        dx = (shape.part(45).x - shape.part(36).x) # note: right - right
        dy = (shape.part(45).y - shape.part(36).y)

        angle = math.atan2(dy,dx) * 180. / math.pi # 计算角度
        RotateMatrix = cv2.getRotationMatrix2D(eye_center, angle, scale=1) # 计算仿射矩阵
        RotImg = cv2.warpAffine(face, RotateMatrix, (face.shape[0], face.shape[1])) # 进行放射变换，即旋转
        faces_aligned.append(RotImg)
    return faces_aligned

def feature(path,foces):
    im_raw =cv2.imread(foces).astype('uint8')   
    detector = dlib.get_frontal_face_detector()
    gray = cv2.cvtColor(im_raw, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    src_faces = []
    for (i, rect) in enumerate(rects):
        (x, y, w, h) = rect_to_bb(rect)
        detect_face = im_raw[y:y+h,x:x+w]
        src_faces.append(detect_face)
        cv2.rectangle(im_raw, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(im_raw, "Face: {}".format(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    faces_aligned = face_alignment(src_faces)
    #cv2.imshow("src", im_raw)    
    for j in os.listdir(path):                 #清空拟装合影照片中分离人脸目录中的文件
        os.remove(path+'\\'+j)         
    i = 0
    for face in faces_aligned:
        #cv2.imshow("det_{}".format(i), face)
        i = i + 1        
        io.imsave(path+'\\'+'Face{}.jpg'.format(i),face)
    #cv2.imshow("Output", im_raw)
    cv2.waitKey(0)

def AipFaceRecognition(pathfile):               
    with open(pathfile,"rb") as f:  
        # b64encode是编码
        base64_data = base64.b64encode(f.read())
    image = str(base64_data,'utf-8')
    imageType = "BASE64"  #3种"URL"，"FACE_TOKEN"
    groupIdList = "Actor"      #你上传百度人脸库照片,用户组ID名叫"qq"
     #""" 调用人脸搜索 """
    a=client.search(image, imageType,groupIdList)  
    #print(a['user_list'][2][ 'user_id'],a['user_list'][3][ 'score'])
    return a    #['result']['user_list'][0]['user_id']

if __name__ == "__main__":
    focePath=r'C:\Users\Administrator\Desktop\face\heying\heying.jpg'   #合影照片人不能超10人
     #"""合影照片中分离人脸子目录"""
    path = r'C:\Users\Administrator\Desktop\face\fenli' 
    APP_ID = '20201374'
    API_KEY = 'XX'
    SECRET_KEY = 'XX'
    client = AipFace(APP_ID, API_KEY, SECRET_KEY)
    feature(path,focePath)
    userlist=[]
    for i in os.listdir(path):
        pathfile=path+'\\'+i
        A=AipFaceRecognition(pathfile)
        if A.get('result',None) !=None:
            if A['result']['user_list'][0]['score']>50:    #取相似值大于50%
                #print('{}照片同{}相似度为{}'.format(i,A['result']['user_list'][0]['user_id'],A['result']['user_list'][0]['score']))
                print('{}照片同{}相似'.format(i,A['result']['user_list'][0]['user_id']))

Face1.jpg照片同huge相似


In [1]:
from aip import AipFace
import base64
import json
import glob
APP_ID = '20201374'
API_KEY = 'XX'
SECRET_KEY = 'XX'
client = AipFace(APP_ID, API_KEY, SECRET_KEY)
for fname in glob.glob("C:/Users/Administrator/Desktop/test/*",recursive=True):
    #print(fname)
    with open(fname,"rb") as f:
        base64_data = base64.b64encode(f.read())
        image = str(base64_data, 'utf-8')
        imageType = "BASE64"
        groupId="Actor"
        #userId="huge"
        #client.addUser(image, imageType, groupId, userId) 
        data=client.search(str(image), imageType, groupId)
        #print(data)
        if data["error_code"]==0:
            print("初步识别为 "+data["result"]["user_list"][0]["user_id"]," 准确度为 "+str(data["result"]["user_list"][0]["score"])+" "+fname)
        else:
            print("图片未识别",fname)

初步识别为 kecheng  准确度为 71.363403320312 C:/Users/Administrator/Desktop/test\kecheng-000.jpg
初步识别为 kecheng  准确度为 68.512046813965 C:/Users/Administrator/Desktop/test\kecheng-001.jpg
初步识别为 kecheng  准确度为 93.972145080566 C:/Users/Administrator/Desktop/test\kecheng-002.jpg
初步识别为 kecheng  准确度为 90.035041809082 C:/Users/Administrator/Desktop/test\kecheng-003.jpg
图片未识别 C:/Users/Administrator/Desktop/test\kecheng-004.jpg
图片未识别 C:/Users/Administrator/Desktop/test\kecheng-005.jpg
图片未识别 C:/Users/Administrator/Desktop/test\kecheng-006.jpg
初步识别为 kecheng  准确度为 95.813140869141 C:/Users/Administrator/Desktop/test\kecheng-007.jpg
初步识别为 kecheng  准确度为 87.771507263184 C:/Users/Administrator/Desktop/test\kecheng-008.jpg
图片未识别 C:/Users/Administrator/Desktop/test\kecheng-009.jpg
图片未识别 C:/Users/Administrator/Desktop/test\kecheng-010.jpg
初步识别为 kecheng  准确度为 90.93660736084 C:/Users/Administrator/Desktop/test\kecheng-011.jpg
初步识别为 zhangge  准确度为 33.383182525635 C:/Users/Administrator/Desktop/test\kecheng-012.jpg
图片未识别 C

初步识别为 zhangge  准确度为 95.470703125 C:/Users/Administrator/Desktop/test\zhangge-052.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-053.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-054.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-055.jpg
初步识别为 zhangge  准确度为 96.276718139648 C:/Users/Administrator/Desktop/test\zhangge-056.jpg
初步识别为 zhangge  准确度为 76.684265136719 C:/Users/Administrator/Desktop/test\zhangge-057.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-058.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-059.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-060.jpg
初步识别为 zhangge  准确度为 96.517654418945 C:/Users/Administrator/Desktop/test\zhangge-061.jpg
初步识别为 zhangge  准确度为 92.320518493652 C:/Users/Administrator/Desktop/test\zhangge-062.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-063.jpg
图片未识别 C:/Users/Administrator/Desktop/test\zhangge-064.jpg
初步识别为 zhangge  准确度为 87.597923278809 C:/Users/Administrator/Desktop/test\zhangge-065.jpg
初步识别为 zhang